# Analysis of supervised models

In [ ]:
import numpy as np
import pandas as pd
import respiration.utils as utils

signals_dir = utils.dir_path('outputs', 'signals')

ground_truth_file = utils.join_paths(signals_dir, 'respiration_gt.csv')
ground_truth = pd.read_csv(ground_truth_file)
ground_truth['signal'] = ground_truth['signal'].apply(eval).apply(np.array)

predictions_file = utils.join_paths(signals_dir, 'pretrained_predictions.csv')
predictions = pd.read_csv(predictions_file)
predictions['signal'] = predictions['signal'].apply(eval).apply(np.array)
predictions['duration'] = pd.to_timedelta(predictions['duration'])
predictions.head()

## Compare the predictions with the ground truth

In [ ]:
import os
from tqdm.auto import tqdm
import respiration.analysis as analysis

analysis_dir = utils.dir_path('outputs', 'analysis', mkdir=True)
analysis_file = utils.join_paths(analysis_dir, 'pretrained_analysis.csv')

if os.path.exists(analysis_file):
    analysis_results = pd.read_csv(analysis_file)
else:
    analysis_results = []

    for index, prediction in tqdm(predictions.iterrows(), total=len(predictions)):
        subject, setting = prediction['subject'], prediction['setting']

        # Get the ground truth signal and sampling rate
        gt_signal = ground_truth[
            (ground_truth['subject'] == subject) &
            (ground_truth['setting'] == setting)
            ]['signal'].values[0]

        predicted_signal = prediction['signal']

        # Check if model is big_small
        if prediction['model'] == 'big_small':
            # Use the cumulative sum of the waveform to get the signal
            predicted_signal = np.cumsum(predicted_signal)

        # Cut the ground truth signal to the same length as the prediction
        gt_signal = gt_signal[:len(predicted_signal)]

        comparator = analysis.SignalComparator(
            predicted_signal,
            gt_signal,
            prediction['sampling_rate'],
        )

        for metric, result in comparator.all_results().items():
            analysis_results.append({
                'subject': subject,
                'setting': setting,
                'model': prediction['model'],
                'metric': metric,
                'prediction': result['prediction'],
                'ground_truth': result['ground_truth'],
            })

    analysis_results = pd.DataFrame(analysis_results)
    analysis_results.to_csv(analysis_file, index=False)

In [ ]:
analysis_results.head()